FAİSS (Facebook AI Similarity Search)

In [27]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

loader = TextLoader("speech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
docs = text_splitter.split_documents(documents)


In [28]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [29]:
embeddings = OllamaEmbeddings()
db = FAISS.from_documents(docs, embeddings)
db

In [30]:
# Querying
query = "How does the speaker describe the desired outcome of the war ?"

docs=db.similarity_search(query)
docs[0].page_content

'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [31]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [32]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='6b6c578c-e11b-4e51-8b31-f0953be60e8c', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
  np.float32(15729.639)),
 (Document(id='fe2736d8-09fd-4b55-98d1-e9d2992663d9', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we 

In [33]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[1.4769623279571533,
 -3.815232753753662,
 2.123159170150757,
 0.6816776394844055,
 -0.15307506918907166,
 1.5119017362594604,
 1.7358057498931885,
 1.1094008684158325,
 -0.6466353535652161,
 -1.2979001998901367,
 2.215670108795166,
 -0.5191461443901062,
 -0.8250889182090759,
 0.6760360598564148,
 -0.1052633747458458,
 -1.2389742136001587,
 -0.5475755333900452,
 0.7615329623222351,
 1.920419454574585,
 -0.9937217831611633,
 -0.4069210886955261,
 -0.4933839738368988,
 1.7680386304855347,
 -3.004695415496826,
 2.009502649307251,
 -1.2058180570602417,
 0.3771815598011017,
 -1.7760913372039795,
 -0.028237750753760338,
 -1.6783950328826904,
 1.9808683395385742,
 -3.5834202766418457,
 -2.257200002670288,
 5.731595039367676,
 2.213743209838867,
 -5.089103698730469,
 -1.0458195209503174,
 1.8553388118743896,
 -1.0564769506454468,
 0.3080158829689026,
 -0.02793685905635357,
 -2.7612483501434326,
 1.662429928779602,
 0.24579080939292908,
 0.835380494594574,
 0.3146989643573761,
 0.27879568934440

In [34]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='6b6c578c-e11b-4e51-8b31-f0953be60e8c', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='fe2736d8-09fd-4b55-98d1-e9d2992663d9', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things w

In [35]:
### Saving And Loading
db.save_local("faiss_index")

In [36]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [37]:
docs

[Document(id='6b6c578c-e11b-4e51-8b31-f0953be60e8c', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='fe2736d8-09fd-4b55-98d1-e9d2992663d9', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things w